# DSCagg Calculation Example for HNTS-MRG 2024

This notebook demonstrates how to calculate the evaluation metric (aggregated Dice Similarity coefficient - DSCagg) for [HNTS-MRG 2024 Challenge](https://hntsmrg24.grand-challenge.org/). More information on the evaluation can be found [here](https://hntsmrg24.grand-challenge.org/tasks-and-evaluation/). The evaluation functions are encapsulated in a Docker container image which will be run on the outputs of participants' submitted algorithms. 

This specific example in this notebook uses a subset of masks from the HNTS-MRG 2024 training dataset, available on [Zenodo](https://zenodo.org/records/11199559). We use 20 out of 150 training case masks for this example (just as a proof of concept).

Credit to the HECKTOR 2022 organizers; most of this code is directly based on their [GitHub implementations](https://github.com/voreille/hecktor/blob/master/notebooks/evaluate_segmentation2022.ipynb).

## Table of Contents

1. [Imports](#Imports)
2. [Functions](#Functions)
3. [DSCagg Calculation](#dscagg-calculation)
4. [Extra: Conventional DSC Calculation](#extra-conventional-dsc-calculation)

## Imports

In [ ]:
import SimpleITK as sitk
import os
import numpy as np
import json

In [ ]:
# Adjust these paths and dataset name as needed
output_json_path = '/homes/nmoradi/test/Dice_summary.json'  
dataset_name = 'midRT_medNext_517'

prediction_folder = '/homes/nmoradi/test/evaluation/predicted_midRT_517_MedNext/predictions_all_hntsmrg_format'
groundtruth_folder = '/homes/nmoradi/test/evaluation/ground_truth_midRT'

## Functions

In [ ]:
def compute_volumes(im):
    """
    Compute the volumes of the GTVp and the GTVn
    """
    spacing = im.GetSpacing()
    voxvol = spacing[0] * spacing[1] * spacing[2]
    stats = sitk.LabelStatisticsImageFilter()
    stats.Execute(im, im)
    nvoxels1 = stats.GetCount(1)
    nvoxels2 = stats.GetCount(2)
    return nvoxels1 * voxvol, nvoxels2 * voxvol

def compute_agg_dice(intermediate_results):
    """
    Compute the aggregate dice score from the intermediate results
    """
    aggregate_results = {}
    TP1s = [v["TP1"] for v in intermediate_results]
    TP2s = [v["TP2"] for v in intermediate_results]
    vol_sum1s = [v["vol_sum1"] for v in intermediate_results]
    vol_sum2s = [v["vol_sum2"] for v in intermediate_results]
    DSCagg1 = 2 * np.sum(TP1s) / np.sum(vol_sum1s)
    DSCagg2 = 2 * np.sum(TP2s) / np.sum(vol_sum2s)
    aggregate_results['AggregatedDsc'] = {
        'GTVp': DSCagg1,
        'GTVn': DSCagg2,
        'mean': np.mean((DSCagg1, DSCagg2)),
    }
    return aggregate_results

def get_intermediate_metrics(patient_ID, groundtruth, prediction):
    """
    Compute intermediate metrics for a given groundtruth and prediction.
    These metrics are used to compute the aggregate dice.
    """
    overlap_measures = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures.SetNumberOfThreads(1)
    overlap_measures.Execute(groundtruth, prediction)

    DSC1 = overlap_measures.GetDiceCoefficient(1)
    DSC2 = overlap_measures.GetDiceCoefficient(2)

    vol_gt1, vol_gt2 = compute_volumes(groundtruth)
    vol_pred1, vol_pred2 = compute_volumes(prediction)

    vol_sum1 = vol_gt1 + vol_pred1
    vol_sum2 = vol_gt2 + vol_pred2
    TP1 = DSC1 * (vol_sum1) / 2
    TP2 = DSC2 * (vol_sum2) / 2
    return {
        "PatientID": patient_ID, # added patient ID so we can pinpoint exact results if needed
        "TP1": TP1,
        "TP2": TP2,
        "vol_sum1": vol_sum1,
        "vol_sum2": vol_sum2,
        "DSC1": DSC1,
        "DSC2": DSC2,
        "vol_gt1": vol_gt1, # needed if you want to exclude empty ground truths in conventional DSC calcs
        "vol_gt2": vol_gt2, 
    }

def resample_prediction(groundtruth, prediction):
    """
    Resample the prediction to the groundtruth physical domain
    """
    resample = sitk.ResampleImageFilter()
    resample.SetSize(groundtruth.GetSize())
    resample.SetOutputDirection(groundtruth.GetDirection())
    resample.SetOutputOrigin(groundtruth.GetOrigin())
    resample.SetOutputSpacing(groundtruth.GetSpacing())
    resample.SetInterpolator(sitk.sitkNearestNeighbor)
    return resample.Execute(prediction) 

def check_prediction(patient_ID, groundtruth, prediction):
    """
    Check if the prediction is valid and apply padding if needed
    """

    # Cast to the same type
    caster = sitk.CastImageFilter()
    caster.SetOutputPixelType(sitk.sitkUInt8)
    caster.SetNumberOfThreads(1)
    groundtruth = caster.Execute(groundtruth)
    prediction = caster.Execute(prediction)

    # Check labels
    stats = sitk.LabelStatisticsImageFilter()
    stats.Execute(prediction, prediction)
    labels = stats.GetLabels()
    if not all([l in [0, 1, 2] for l in labels]):
        raise RuntimeError(
            f"Patient {patient_ID}: Error. The labels are incorrect. The labels should be background: 0, GTVp: 1, GTVn: 2."
        )
    # Check spacings
    if not np.allclose(
            groundtruth.GetSpacing(), prediction.GetSpacing(), atol=0.000001):
        raise RuntimeError(
            f"Patient {patient_ID}: Error. The resolution of the prediction is different from the MRI ground truth resolution."
        )

    # Check if resampling is needed
    needs_resampling = False
    if prediction.GetSize() != groundtruth.GetSize():
        needs_resampling = True
    elif not np.allclose(prediction.GetDirection(), groundtruth.GetDirection(), atol=0.000001):
        needs_resampling = True
    elif not np.allclose(prediction.GetOrigin(), groundtruth.GetOrigin(), atol=0.000001):
        needs_resampling = True

    if needs_resampling:
        print(f"Patient {patient_ID}: Prediction checked, resampling prediction to match ground truth...")
        prediction = resample_prediction(groundtruth, prediction)
    else:
        print(f'Patient {patient_ID}: Prediction checked, everything correct and no resampling needed.')
        # To be sure that sitk won't trigger unnecessary errors
        prediction.SetSpacing(groundtruth.GetSpacing())

    return prediction

## DSCagg Calculation

Remember DSCagg is calculated over the entire set of data so you do not get patient-level datapoints like with conventional volumetric DSC.

Ground truth masks here are the mid-RT masks while the "prediction" masks here are the registered pre-RT masks. 

In [ ]:
# first set up the ground truth and prediction paths

prediction_files = [os.path.join(prediction_folder, file) for file in os.listdir(prediction_folder) if "nii.gz" in file]
groundtruth_files = [os.path.join(groundtruth_folder, file) for file in os.listdir(groundtruth_folder) if "nii.gz" in file]

print("Prediction files", prediction_files, "\n")

print("Ground truth files", groundtruth_files)

Prediction files ['/homes/nmoradi/test/evaluation/predicted_midRT_517_MedNext/predictions_all_hntsmrg_format/145_midRT_mask.nii.gz', '/homes/nmoradi/test/evaluation/predicted_midRT_517_MedNext/predictions_all_hntsmrg_format/150_midRT_mask.nii.gz', '/homes/nmoradi/test/evaluation/predicted_midRT_517_MedNext/predictions_all_hntsmrg_format/41_midRT_mask.nii.gz', '/homes/nmoradi/test/evaluation/predicted_midRT_517_MedNext/predictions_all_hntsmrg_format/75_midRT_mask.nii.gz', '/homes/nmoradi/test/evaluation/predicted_midRT_517_MedNext/predictions_all_hntsmrg_format/60_midRT_mask.nii.gz', '/homes/nmoradi/test/evaluation/predicted_midRT_517_MedNext/predictions_all_hntsmrg_format/171_midRT_mask.nii.gz', '/homes/nmoradi/test/evaluation/predicted_midRT_517_MedNext/predictions_all_hntsmrg_format/164_midRT_mask.nii.gz', '/homes/nmoradi/test/evaluation/predicted_midRT_517_MedNext/predictions_all_hntsmrg_format/133_midRT_mask.nii.gz', '/homes/nmoradi/test/evaluation/predicted_midRT_517_MedNext/predi

Please note in the below code some warnings (LabelOverlapMeasuresImageFilter) are thrown because there is no label in the ground truth file and the prediction file. This is not an error and expected behavior for the given setup.

In [5]:
results = list()
for f in prediction_files:
    patient_ID = os.path.split(f)[-1].split('_')[0] # get the patient ID from the path 
    print(patient_ID)
    gt_file = [k for k in groundtruth_files if os.path.split(k)[-1].split('_')[0] == patient_ID][0]

    print(f"Evaluating patient {patient_ID}")

    prediction = sitk.ReadImage(str(f))
    groundtruth = sitk.ReadImage(str(gt_file))
    prediction = check_prediction(patient_ID, groundtruth, prediction) 


    results.append(get_intermediate_metrics(patient_ID, groundtruth, prediction))

145
Evaluating patient 145
Patient 145: Prediction checked, everything correct and no resampling needed.


150
Evaluating patient 150
Patient 150: Prediction checked, everything correct and no resampling needed.
41
Evaluating patient 41
Patient 41: Prediction checked, everything correct and no resampling needed.
75
Evaluating patient 75
Patient 75: Prediction checked, everything correct and no resampling needed.
60
Evaluating patient 60
Patient 60: Prediction checked, everything correct and no resampling needed.
171
Evaluating patient 171
Patient 171: Prediction checked, everything correct and no resampling needed.


LabelOverlapMeasuresImageFilter (0x56144d43c6e0): Label  not found.



164
Evaluating patient 164
Patient 164: Prediction checked, everything correct and no resampling needed.


LabelOverlapMeasuresImageFilter (0x56144d43c6e0): Label  not found.



...

39
Evaluating patient 39
Patient 39: Prediction checked, everything correct and no resampling needed.
91
Evaluating patient 91
Patient 91: Prediction checked, everything correct and no resampling needed.
136
Evaluating patient 136
Patient 136: Prediction checked, everything correct and no resampling needed.


LabelOverlapMeasuresImageFilter (0x56144d43c6e0): Label  not found.



18
Evaluating patient 18
Patient 18: Prediction checked, everything correct and no resampling needed.


LabelOverlapMeasuresImageFilter (0x56144d43c6e0): Label  not found.



27
Evaluating patient 27
Patient 27: Prediction checked, everything correct and no resampling needed.
109
Evaluating patient 109
Patient 109: Prediction checked, everything correct and no resampling needed.
32
Evaluating patient 32
Patient 32: Prediction checked, everything correct and no resampling needed.
3
Evaluating patient 3
Patient 3: Prediction checked, everything correct and no resampling needed.


Display aggregated DSC metrics. This is what will be used in the challenge evaluation/ranking.

In [ ]:
# Display raw results
print("The raw results are:", results, "\n")

# Compute and display aggregate dice scores
agg_dice_scores = compute_agg_dice(results)
print(f"Aggregate dice scores: {agg_dice_scores}\n")

The raw results are: [{'PatientID': '145', 'TP1': 760.0, 'TP2': 2649.5, 'vol_sum1': 3582.5, 'vol_sum2': 7218.0, 'DSC1': 0.424284717376134, 'DSC2': 0.7341368800221668, 'vol_gt1': 2738.0, 'vol_gt2': 3141.0}, {'PatientID': '150', 'TP1': 1093.0, 'TP2': 10777.0, 'vol_sum1': 7511.5, 'vol_sum2': 25254.0, 'DSC1': 0.29102043533249017, 'DSC2': 0.853488556268314, 'vol_gt1': 1998.0, 'vol_gt2': 12366.0}, {'PatientID': '41', 'TP1': 6464.000000000001, 'TP2': 837.0000000000001, 'vol_sum1': 20052.0, 'vol_sum2': 1960.5, 'DSC1': 0.644723718332336, 'DSC2': 0.8538638102524867, 'vol_gt1': 11849.5, 'vol_gt2': 1027.5}, {'PatientID': '75', 'TP1': 5606.500000000001, 'TP2': 2755.5, 'vol_sum1': 13894.5, 'vol_sum2': 6242.0, 'DSC1': 0.807009967972939, 'DSC2': 0.8828900993271387, 'vol_gt1': 6652.5, 'vol_gt2': 3090.5}, {'PatientID': '60', 'TP1': 3236.0, 'TP2': 12814.499999999998, 'vol_sum1': 9403.5, 'vol_sum2': 29053.5, 'DSC1': 0.688254373371617, 'DSC2': 0.8821312406422633, 'vol_gt1': 5589.5, 'vol_gt2': 14794.0}, {'P

## Extra: Conventional DSC Calculation

Since conventional volumetric DSC was also calculated during the DSCagg calculation, we can also display these values as well just for reference. These metrics will not be used in the challenge directly but may be handy to know.

In [ ]:
# Extract DSC1 and DSC2 values
DSC1_values = [result["DSC1"] for result in results]
DSC2_values = [result["DSC2"] for result in results]

# Compute and display mean DSC1 and DSC2
mean_DSC1 = np.mean(DSC1_values)
mean_DSC2 = np.mean(DSC2_values)
std_DSC2 = np.std(DSC2_values)
std_DSC1 = np.std(DSC1_values)

print(f"Mean DSC1 (GTVp): {mean_DSC1}")
print(f"Mean DSC2 (GTVn): {mean_DSC2}\n")
print(f"STD DSC1 (GTVp): {std_DSC1}\n")
print(f"STD DSC2 (GTVn): {std_DSC2}\n")

Mean DSC1 (GTVp): 0.41460883182707203
Mean DSC2 (GTVn): 0.676514389776771

STD DSC1 (GTVp): 0.31586184287354396

STD DSC2 (GTVn): 0.3064299744756378



Conventional volumetric DSC may be disproportionately affected by a single false negative/postive result (yielding a DSC of 0). Therefore, it may be more informative to remove instances where the ground truth is empty. The code below removes instances with empty ground truth before computing the mean DSC values.

Alternativley, one could use a smoothing term making the DSC of a case without ground truth equal to 1 if there is no false positive, 0 otherwise (we don't implement this here). Again this would still potentially overly penalize false positives (1 voxel false positive treated the same as 1000 voxel false positive).

In [ ]:
# Extract non-zero DSC1 and DSC2 values and print removed patient IDs
DSC1_values_nozeros = []
DSC2_values_nozeros = []
removed_patients_DSC1 = []
removed_patients_DSC2 = []

for result in results:
    patient_id = result["PatientID"]
    if result["vol_gt1"] != 0.0:
        DSC1_values_nozeros.append(result["DSC1"])
    else:
        removed_patients_DSC1.append(patient_id)
    if result["vol_gt2"] != 0.0:
        DSC2_values_nozeros.append(result["DSC2"])
    else:
        removed_patients_DSC2.append(patient_id)

# Print removed patient IDs
print("Removed patient IDs with empty ground truth volumes for DSC1:", removed_patients_DSC1)
print("Removed patient IDs with empty ground truth volumes for DSC2:", removed_patients_DSC2, "\n")

# Compute and display mean non-zero DSC1 and DSC2
mean_DSC1_nozeros = np.mean(DSC1_values_nozeros)
mean_DSC2_nozeros = np.mean(DSC2_values_nozeros)
std_DSC2_nozeros = np.std(DSC2_values_nozeros)
std_DSC1_nozeros = np.std(DSC1_values_nozeros)
print(f"Mean DSC1 (GTVp) without empty ground truth: {mean_DSC1_nozeros}")
print(f"Mean DSC2 (GTVn) without empty ground truth: {mean_DSC2_nozeros}\n")
print(f"STD DSC1 (GTVp) without empty ground truth: {std_DSC1_nozeros}\n")
print(f"STD DSC2 (GTVn) without empty ground truth: {std_DSC2_nozeros}\n")

Removed patient IDs with empty ground truth volumes for DSC1: ['190', '107', '29', '94', '101', '196', '188', '177', '148', '47', '52', '165', '95', '191', '139', '118', '25', '86', '46', '187', '105', '48', '152', '56', '45', '154', '71', '129', '21', '5', '34', '42', '174', '8', '136', '18']
Removed patient IDs with empty ground truth volumes for DSC2: ['171', '164', '22', '29', '183', '156', '170', '55', '80', '23', '132', '25', '197', '20', '77', '178', '64', '181', '21', '179', '42'] 

Mean DSC1 (GTVp) without empty ground truth: 0.5455379366145684
Mean DSC2 (GTVn) without empty ground truth: 0.7866446392753151

STD DSC1 (GTVp) without empty ground truth: 0.24463813393019146

STD DSC2 (GTVn) without empty ground truth: 0.15017288765948106



## Saveing the results

In [ ]:
with open(output_json_path, 'r') as file:
    output = json.load(file)

dice_scores = { 
    "Mean DSC1 (GTVp)": mean_DSC1,
    "Mean DSC2 (GTVn)": mean_DSC2, 
    "STD DSC1 (GTVp)": std_DSC1,
    "STD DSC2 (GTVn)": std_DSC2,
    "Mean DSC1 (GTVp) without empty ground truth": mean_DSC1_nozeros,
    "Mean DSC2 (GTVn) without empty ground truth": mean_DSC2_nozeros,
    "STD DSC1 (GTVp) without empty ground truth": std_DSC1_nozeros,
    "STD DSC2 (GTVn) without empty ground truth": std_DSC2_nozeros
}
dice_scores.update(agg_dice_scores)


In [ ]:

new_data = {
    dataset_name : dice_scores
}

output.update(new_data)
with open(output_json_path, 'w') as outfile:
    json.dump(output, outfile, indent=4)